## This part describe how to refine the mix-grained dataset to fine grained dataset

In [15]:
import json
import pickle as pkl
import numpy as np

In [16]:
path = r'/home/shensy/Code/python/doduo/data/ssy_test.coltype.pkl'

In [5]:
with open (path, "rb") as f:
    turl_type_test = pkl.load(f)
print(turl_type_test)

        table_id                                             labels  \
0      6675886-1                      [sports.sports_league_season]   
1      6675886-1                                    [people.person]   
2      6675886-1                        [organization.organization]   
3      6675886-1                      [business.business_operation]   
4      6675886-1                               [automotive.company]   
...          ...                                                ...   
1870  40723972-2  [location.administrative_division, location.lo...   
1871  40723972-2                        [organization.organization]   
1872  40723972-2  [government.political_party, organization.orga...   
1873  40723972-2                        [organization.organization]   
1874  40723972-2  [government.political_party, organization.orga...   

                                                   data  \
0     1998 1999 2000 2001 2002 2003 2004 2005 2006 2...   
1     Jackie Stewart Adrian F

### coarse-grained type like `organization.organization` exist, drop them to ...

In [3]:
exclude_list = ['people.person','location.location','sports.sport','time.event', 'sports.pro_athlete']

In [35]:
del_idx = []
failed_table = []

for idx, row in turl_type_test.iterrows():
    if len(row.labels)>2:
        del_idx.append(idx)
        failed_table.append(row.table_id)
    if row.labels[0] in exclude_list:
        del_idx.append(idx)      
        
for idx, row in turl_type_test.iterrows(): 
    if row.table_id in failed_table:
        del_idx.append(idx)     
        
del_idx = list(set(del_idx))

In [37]:
test_df = turl_type_test.drop(del_idx)

ff = open("../data/fine-grained_WikiTables.pkl","wb")
pkl.dump(test_df, ff)
ff.close

len(test_df)

1361

In [34]:
len(test_df.groupby("table_id"))

622

tv.tv_actor
location.us_state
music.composer
sports.boxer
automotive.company



# Create csv for boss to read 

In [23]:
import json
import csv
import pandas as pd
import numpy as np

In [18]:
# from json to csv
with open('../data/fine-grained_WikiTables.json', 'r', encoding="utf8") as f:
    example = json.load(f)[0]
display(example)
len(example)

['6675886-1',
 'indy japan 300',
 6675886,
 'past winners',
 ' usac championship car history (non-championship, fuji )',
 ['season season', 'chassis chassis', 'engine engine'],
 [[[[2, 0], [5710555, '1998']],
   [[3, 0], [6498428, '1999']],
   [[4, 0], [11056278, '2000']],
   [[5, 0], [10637245, '2001']],
   [[6, 0], [9014994, '2002']],
   [[8, 0], [4485956, '2003']],
   [[9, 0], [4479387, '2004']],
   [[10, 0], [2454550, '2005']],
   [[11, 0], [2696531, '2006']],
   [[12, 0], [6235033, '2007']],
   [[13, 0], [6939922, '2008']],
   [[14, 0], [13512105, '2009']],
   [[15, 0], [16099880, '2010']],
   [[16, 0], [19850806, '2011']]],
  [[[0, 3], [676392, 'Lola']],
   [[2, 3], [30865876, 'Reynard']],
   [[3, 3], [30865876, 'Reynard']],
   [[4, 3], [676392, 'Lola']],
   [[5, 3], [676392, 'Lola']],
   [[6, 3], [676392, 'Lola']],
   [[8, 3], [1226646, 'Dallara']],
   [[9, 3], [1226646, 'Dallara']],
   [[10, 3], [1226646, 'Dallara']],
   [[11, 3], [1226646, 'Dallara']],
   [[12, 3], [1226646, '

8

In [19]:
def list2dataframe(list_table):
    table_idx, page_title, table_id, headline, table_name, header, data, gd_header = list_table
    table_idx=table_idx.split('-')[-1]
    list_data=[]
    for row in data:
        temp_row=[]
        for cell in row:
            temp_row.append(cell[-1][-1]) 
        list_data.append(temp_row)

    list_data = map(list, zip(*list_data))
    df=pd.DataFrame(list_data, columns = header)
    return table_idx, page_title, table_id, headline, table_name, header, df, gd_header

In [24]:
with open('../data/fine-grained_WikiTables.json', 'r', encoding="utf8") as f:
    tables = json.load(f)
    
for tab in tables:
    table_idx, page_title, table_id, headline, table_name, header, df, gd_header = list2dataframe(tab)
    gd = []
    for h in gd_header:
        gd += h
    
    int_tab = [["***PAGE TITLE***"],[page_title],
               ["***HEADLINE***"], [headline], 
               ["***TABLE NAME***"],[table_name],
               ["***GROUND TRUTH***"],gd, 
               ["***HEADER***"],header,["***TABLE CONTENT***"]] + np.array(df).tolist()+[[]]
    print(int_tab)
    with open('../data/fine-grined_WikiTables.csv', "a", encoding='utf8') as f:
        writer = csv.writer(f)
        writer.writerows(int_tab)



[['indy japan 300'], ['past winners'], [' usac championship car history (non-championship, fuji )'], ['sports.sports_league_season', 'business.business_operation', 'automotive.company'], ['season season', 'chassis chassis', 'engine engine'], ['1998', 'Lola', 'Ford'], ['1999', 'Reynard', 'Ford - Cosworth'], ['2000', 'Reynard', 'Ford - Cosworth'], ['2001', 'Lola', 'Ford - Cosworth'], ['2002', 'Lola', 'Ford - Cosworth'], ['2003', 'Lola', 'Toyota'], ['2004', 'Dallara', 'Toyota'], ['2005', 'Dallara', 'Honda'], ['2006', 'Dallara', 'Honda'], ['2007', 'Dallara', 'Honda'], ['2008', 'Dallara', 'Honda'], ['2009', 'Dallara', 'Honda'], ['2010', 'Dallara', 'Honda'], ['2011', 'Dallara', 'Honda']]
